# LAB: Pipelines y Transformadores


## 1. Introducción
En este Lab vamos a crear Pipelines para pre-procesar datos y extraer características sobre el [Titanic dataset](http://www.kaggle.com/c/titanic-gettingStarted/data).

El dataset es una lista de pasajeros del trasatlántico más famoso. La segunda columna del dataset ("survived") indica si la persona ha sobrevivido (1) o no (0) al naufragio. El resto de las columnas contienen información diversa sobre cada uno de los pasajeros.

* Levantamos el dataset (Titanic.csv) en un DataFrame.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

df = pd.read_csv('Titanic.csv')
df.head()

Vamos a hacer una primera exploración de los datos:

- ¿Hay features numéricas?
- ¿Hay features categóricas?
- ¿Hay datos incompletos? ¿En qué columnas?
- ¿Cuáles te parecen importantes para rellenar?

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

# 2. Preprocesando cada grupo de columnas

Observamos que el preprocesamiento de los datos requiere distintos enfoques para distintos tipos de columnas: algunas requieren imputación, otras requieren generar variables Dummies y otras sería conveniente estandarizarlas.

La idea es armar un pipeline separado para el preprocesamiento que necesita cada grupo de variables y luego unirlos todos con el método make_union() que ejecutará todos los pipelines para luego concatenar el resultado.

Para hacer las transformaciones de cada grupo de columnas sugerimos crear un transformer de sklearn ColumnSelector que permita seleccionar un grupo de columnas del DataFrame donde queremos aplicar las transformaciones.



## 2.1 Edad

Se puede observar que hay varios pasajeros sin información de edad (columna "Age"). Vamos a intentar llenar los datos de esta columna. Exploremos la distribución de valores para los datos existentes y pensemos una estrategia.

In [ ]:
df.Age.plot(kind = 'hist');

#### Transformador de Edad

Dependiendo la estrategia que hayamos decido vamos a necesitar imputar los datos de edad faltantes, ya sea usando un transformador del módulo de pre-procesamiento o crear un transformador custom transformer.
Esto podría implicar:

- Llenar los datos faltantes
- Escalar los valores de Edad

¿Qué clases de sklearn permiten imputar datos y llenar valores faltantes?

In [ ]:
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            return pd.DataFrame(X[self.columns])
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self

In [ ]:
cs = ColumnSelector('Age')
cs.transform(df).sample(3)

In [ ]:
from sklearn.pipeline import make_pipeline, make_union

In [ ]:
age_pipe = make_pipeline(ColumnSelector('Age'),
                         Imputer(),
                         StandardScaler()) 

## 2.2. Variables Categóricas

"Embarked" y "Pclass" son variables categóricas. Usá la función get_dummies de pandas para crear columnas correspondientes a los valores de las mismas.

"Embarked" tiene un par de datos faltantes. Llenalos con el puerto de embarque más común en el dataset.

Sugerencia: Crear un transformador custom que "envuelva" el uso de get_dummies.

In [ ]:
df.Embarked.value_counts()

In [ ]:
df.Embarked = df.Embarked.fillna('S')

In [ ]:
pd.get_dummies(pd.DataFrame(df['Embarked'].head()))

In [ ]:
class GetDummiesTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def transform(self, X, *_):
        if isinstance(X, pd.DataFrame):
            return pd.get_dummies(X[self.columns], columns = self.columns, drop_first = True)
        else:
            raise TypeError("Este Transformador solo funciona en DF de Pandas")
    
    def fit(self, X, *_):
        return self
    


In [ ]:
gdt = GetDummiesTransformer(['Embarked'])
gdt.fit_transform(df.head())

In [ ]:
dummies_pipe = GetDummiesTransformer(['Pclass', 'Embarked', 'Sex'])

dummies_pipe.fit_transform(df).head()

## 2.3  Estandarizar la Tarifa

Escalar el atributo "Fare" (Tarifa) usando uno de los escaladores existentes en el módulo de preprocesamiento. 

In [ ]:
fare_pipe = make_pipeline(ColumnSelector('Fare'),
                          StandardScaler())

fare_pipe.fit_transform(df.head())

## 3. Union

Utilizá una FeatureUnion o la función make_union para combinar todos los pipelines que has creado.

In [ ]:
union = make_union(age_pipe,
                   dummies_pipe,
                   fare_pipe)

union.fit_transform(df.head())

In [ ]:
X = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = df['Survived']

X = union.fit_transform(X)
X

## 4. Predicciones

Ahora utilicemos GridSearch para evaluar la performance de estas transformaciones, seguidas de un modelo SVM. 
Para esto exploren distintos valores de parámetros para C y Gamma.

Hagan un split entre train y test.



In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
pipeline = Pipeline([
   ('union', union), 
   ('svc', SVC()), 
])
parameters = {
    'svc__C': np.linspace(1e-2,1.5, 5),
    'svc__gamma': ['auto',1e-3,1,1.1,1.2],
}

In [ ]:
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

X_train, X_test, y_train, y_test = train_test_split(
    df[columns], df['Survived'], test_size=0.33, random_state=42)

In [ ]:
grid_search = GridSearchCV (pipeline, parameters, n_jobs = -1 , verbose = 2 )

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_) 

print("Best parameters set:" )

best_parameters = grid_search.best_estimator_.get_params()

for param_name in sorted (parameters . keys()):
    print("\t %s: %r" % (param_name, best_parameters[param_name])) 

## 5. Performance sobre datos nuevos

Con el mejor modelo seleccionado mediante cross validation en los datos de entrenamiento, evalúen el accuracy y el classification report sobre datos no observados.


In [ ]:
grid_search.best_estimator_.fit(X_train,y_train)

In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))